### Loading Dataset

In [1]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(source("tree_term_climb.R"))
suppressPackageStartupMessages(library(beaveR))
suppressPackageStartupMessages(library(TreeSummarizedExperiment))

saveDir <- "environment/Chimp/DE"
load(file.path(saveDir, "tseChimp.RData"))
load(file.path(saveDir, "yAll.RData"))
load(file.path(saveDir, "y.RData"))
load(file.path(saveDir, "yg.RData"))
treeCons <- rowTree(tseChimp)
nleaves <- length(treeCons$tip)

# metaData <- read.delim("/fs/cbcb-lab/rob/students/noor/Uncertainity/real_datasets/GSE100505_EDL_MAST/SRR_Acc_List.txt")
# quantDir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/real_datasets/GSE100505_EDL_MAST/sal_out/mode_gcbias=True/posttype=gibbs_npost=100_tf=100"
# samples <- metaData$RUN
# files <- file.path(quantDir, samples, "quant.sf")
# colData <- cbind(data.frame(files = files, names = samples), condition = as.factor(metaData$TissueName))
# seMuscle <- tximeta::tximeta(colData)

In [3]:
sum(mcols(y)[["qvalue"]] < 0.1, na.rm=T)

[1] 11886

In [2]:
treeCons <- rowTree(yAll)
nleaves <- length(treeCons$tip)

#### Transcript level analysis

In [3]:
sum(mcols(y)[["keep"]])
sum(mcols(yAll)[["keep"]])
sum(mcols(y)[["pvalue"]] < 0.1, na.rm=T)
sum(mcols(yAll)[treeCons$tip.label,"pvalue"] < 0.1, na.rm=T)

sum(mcols(y)[["qvalue"]] < 0.1, na.rm=T)
sum(mcols(yAll)[treeCons$tip.label,"qvalue"] < 0.1, na.rm=T)

[1] 25236

[1] 49951

[1] 13831

[1] 13496

[1] 11886

[1] 11813

##### Differentially Expressed Transcripts

In [4]:
sapply(c(0.01,0.05,0.1), function(x) sum(mcols(y)[["qvalue"]] <= x, na.rm=T))
dTxps <- lapply(c(0.01,0.05,0.1), function(x) rownames(y)[which(mcols(y)[["qvalue"]] <= x)])

[1]  3539 10133 11886

##### Genes Mapping to those transcripts

In [5]:
mapDf <- rowData(tseChimp)
txpGenes <- lapply(dTxps, function(txps) unique(unlist(mapDf[txps, "GENEID"])))##Genes that map to transcripts
sapply(txpGenes, length)

[1] 3236 8304 9492

#### Gene Level

##### Differentially expressed genes

In [6]:
dges <- lapply(c(0.01,0.05,0.1), function(x) rownames(gy)[which(mcols(gy)[["qvalue"]] <= x)])
sapply(dges, length)

[1] 3786 6011 7331

### Genes that are considered differentially expressed in gene level analysis but don't contain a single differentially expressed transcript

In [7]:
diffGenes <- lapply(seq_along(dges), function(i) setdiff(dges[[i]], txpGenes[[i]])) ## dges that are missing in genes mapped by differential transcripts
sapply(diffGenes, length)
diffGenes2 <- lapply(seq_along(dges), function(i) setdiff(txpGenes[[i]], dges[[i]])) ## Genes that are mapped to differential transcripts but not mapped by genes
sapply(diffGenes2, length)
# sum(mcols(y)[unlist(mcols(yg)[diffGenes[[2]],"tx_ids"]), "qvalue"] < 0.05, na.rm=T)==0 ## None of the diffGenes should contain a differential transcript

[1] 1173  255  266

[1]  623 2548 2427

#### TreeDE, with the total number of nodes, inner nodes and their height distribution

In [8]:
load(file.path(saveDir, "tAfterBHMoreClimbMIRV.RData"))
treeDE <- tAfterBHMoreClimbMIRV[["mIRV=0.4"]] ## renaming the treeDE variable
sapply(treeDE, length) ## number of differentially expressed nodes
sapply(treeDE, function(nodes) sum(nodes > nleaves)) ## number of inner nodes
lapply(treeDE, function(nodes) table(node.depth(treeCons,2)[nodes])) ## Distribution of node heights

[1]  7340 10378 12054

[1] 2156 2693 2944

[[1]]

   1    2    3    4    5    6    7    8    9 
5184 1294  536  189   86   27   15    4    5 

[[2]]

   1    2    3    4    5    6    7    8    9 
7685 1688  634  229   81   35   17    6    3 

[[3]]

   1    2    3    4    5    6    7    8    9 
9110 1882  683  232   81   38   18    7    3 


##### Genes mapping to the treeDE nodes and their number

In [9]:
genesTreeDE <- lapply(treeDE, function(nodes) {
    lapply(Descendants(treeCons,nodes), function(desc) unique(unlist(mapDf[treeCons$tip[desc],"GENEID"])))})
sapply(genesTreeDE, function(genes) length(unique(unlist(genes)))) ## Number of genes mapping to treeDE nodes
sapply(genesTreeDE, function(nodes) table(sapply(nodes,length))) ## Distribution of number of genes and nodes

[1] 6485 8770 9944

1,7220,10215,11871
2,119,162,182
3,1,1,1


#### Genes that map only to treeDE but are neither DE or covered by differential transcripts

In [10]:
innerNodesUniqueGenes <- lapply(seq_along(genesTreeDE), function(i) setdiff(unlist(genesTreeDE[[i]]), union(dges[[i]], txpGenes[[i]])))
head(innerNodesUniqueGenes[[2]]) 
sapply(innerNodesUniqueGenes, length)

[1] "ENSPTRG00000019904" "ENSPTRG00000050773" "ENSPTRG00000019837"
[4] "ENSPTRG00000043336" "ENSPTRG00000016440" "ENSPTRG00000003776"

[1] 2166  395  362

In [13]:
unGenesSNode <- lapply(seq_along(innerNodesUniqueGenes), function(i) {
        gNodes <- innerNodesUniqueGenes[[i]]
        gNodes[sapply(gNodes, function(gene) {
    sum(sapply(genesTreeDE[[i]], function(g) sum(gene %in% g) > 0 & length(g)>1)) == 0 ##gene maps unique to a single node
})]
})
# innerNodesUniqueGenes[[2]][sapply(innerNodesUniqueGenes[[2]], function(gene) {
#     sum(sapply(genesTreeDE[[2]], function(g) sum(gene %in% g) > 0 & length(g)>1)) == 0
# })]

In [14]:
sapply(unGenesSNode, length) ### Nodes that map to a single gene

[1] 1992  217  175

Such inner nodes can be possible DTUs. We thus extract the treeDE nodes to which these genes map to and try to see if there are multiple such tree DE nodes that map to same gene.Looking at the 0.05 threshold change, we find only 0 gene for which a dtu on the inner nodes is observed.

For the remaining that have map to inner node, infRV decreases compared to their children though the logFC is lower compared to children. logFC though definitely decreases at gene.

In [15]:
qval <- 0.15
i <- 2
gN <- unGenesSNode[[i]][which(mcols(gy)[unGenesSNode[[i]],"qvalue"] > qval)]
length(gN) ## genes belonging to the unique treeDE nodes that have qvalue > 0.15 when doing gene level DE analysis

# extracting genes unique to treeDE and have logFC > 0.2 or have atleast two sig nodes
twoNodes <- list() 
lfcNodes <- c()
j <- 1
for(g in gN) {
    iid <- which(sapply(genesTreeDE[[i]], function(genes) sum(g %in%  genes) > 0))
    if(length(iid) > 1) {
        twoNodes[[j]] <- iid
        j <- j + 1
    }
    else if(length(which(abs(mcols(yAll)[treeDE[[2]][iid],"log2FC"]) > 0.2)) > 0) {
        lfcNodes <- c(lfcNodes, iid)
    }
}
length(twoNodes)
length(lfcNodes)

[1] 106

[1] 0

[1] 101

##### Other example of the above genes that however show no dtu

In [20]:
head(order(mcols(yAll)[treeDE[[2]][lfcNodes],"qvalue"]), 30)

[1]   7  60 101  17  80  71  43  12  59  42  79  32  74   5  34  63  65  40  45
[20]  51   1  41  91  99  15   4  83  16   8  86

[1] "ENSPTRG00000003756"


In [16]:
tnames <- sapply(strsplit(rownames(y), ".", fixed=T), function(g) g[1])
rnames <- rownames(y)
names(rnames) <- tnames

In [58]:
## A lowering of the LFC but also along with a substantial decrease in infRV
## Some of these nodes map to multiple genes some of which are DEs

### Show 10, 96, 129, 60
j <- 7
gs <- genesTreeDE[[2]][[lfcNodes[j]]]
print(gs)
mcols(gy)[gs,c("log10mean", "log2FC", "qvalue")]
mcols(yAll)[treeDE[[2]][lfcNodes[j]],c("meanInfRV", "log10mean", "log2FC", "qvalue")]
txps <- rnames[unlist(rowData(gy)[gs,"tx_ids"])]
mcols(y)[txps, c("meanInfRV", "log10mean", "log2FC", "qvalue")]
treeCons$tip[unlist(Descendants(treeCons, treeDE[[2]][lfcNodes[j]]))]

j <- 60
gs <- genesTreeDE[[2]][[lfcNodes[j]]]
print(gs)
mcols(gy)[gs,c("log10mean", "log2FC", "qvalue")]
mcols(yAll)[treeDE[[2]][lfcNodes[j]],c("meanInfRV", "log10mean", "log2FC", "qvalue")]
txps <- rnames[unlist(rowData(gy)[gs,"tx_ids"])]
mcols(y)[txps, c("meanInfRV", "log10mean", "log2FC", "qvalue")]
treeCons$tip[unlist(Descendants(treeCons, treeDE[[2]][lfcNodes[j]]))]


j <- 101
gs <- genesTreeDE[[2]][[lfcNodes[j]]]
print(gs)
mcols(gy)[gs,c("log10mean", "log2FC", "qvalue")]
mcols(yAll)[treeDE[[2]][lfcNodes[j]],c("meanInfRV", "log10mean", "log2FC", "qvalue")]
txps <- rnames[unlist(rowData(gy)[gs,"tx_ids"])]
mcols(y)[txps, c("meanInfRV", "log10mean", "log2FC", "qvalue")]
treeCons$tip[unlist(Descendants(treeCons, treeDE[[2]][lfcNodes[j]]))]

j <- 17
gs <- genesTreeDE[[2]][[lfcNodes[j]]]
print(gs)
mcols(gy)[gs,c("log10mean", "log2FC", "qvalue")]
mcols(yAll)[treeDE[[2]][lfcNodes[j]],c("meanInfRV", "log10mean", "log2FC", "qvalue")]
txps <- rnames[unlist(rowData(gy)[gs,"tx_ids"])]
mcols(y)[txps, c("meanInfRV", "log10mean", "log2FC", "qvalue")]
treeCons$tip[unlist(Descendants(treeCons, treeDE[[2]][lfcNodes[j]]))]


# j <- 140
# gs <- genesTreeDE[[2]][[lfcNodes[j]]]
# print(gs)
# mcols(yg)[gs,c("log10mean", "log2FC", "qvalue")]
# mcols(yAll)[treeDE[[2]][lfcNodes[j]],c("meanInfRV", "log10mean", "log2FC", "qvalue")]
# txps <- intersect(unlist(rowData(yg)[gs,"tx_ids"]), rownames(y))
# mcols(y)[txps, c("meanInfRV", "log10mean", "log2FC", "qvalue")]
# treeCons$tip[unlist(Descendants(treeCons, treeDE[[2]][lfcNodes[j]]))]

# j <- 6
# gs <- genesTreeDE[[2]][[lfcNodes[j]]]
# print(gs)
# mcols(yg)[gs,c("log10mean", "log2FC", "qvalue")]
# mcols(yAll)[treeDE[[2]][lfcNodes[j]],c("meanInfRV", "log10mean", "log2FC", "qvalue")]
# txps <- intersect(unlist(rowData(yg)[gs,"tx_ids"]), rownames(y))
# mcols(y)[txps, c("meanInfRV", "log10mean", "log2FC", "qvalue")]
# treeCons$tip[unlist(Descendants(treeCons, treeDE[[2]][lfcNodes[j]]))]

# j <- 144
# gs <- genesTreeDE[[2]][[lfcNodes[j]]]
# print(gs)
# mcols(yg)[gs,c("log10mean", "log2FC", "qvalue")]
# mcols(yAll)[treeDE[[2]][lfcNodes[j]],c("meanInfRV", "log10mean", "log2FC", "qvalue")]
# txps <- intersect(unlist(rowData(yg)[gs,"tx_ids"]), rownames(y))
# mcols(y)[txps, c("meanInfRV", "log10mean", "log2FC", "qvalue")]
# treeCons$tip[unlist(Descendants(treeCons, treeDE[[2]][lfcNodes[j]]))]

# j <- 77
# gs <- genesTreeDE[[2]][[lfcNodes[j]]]
# print(gs)
# mcols(yg)[gs,c("log10mean", "log2FC", "qvalue")]
# mcols(yAll)[treeDE[[2]][lfcNodes[j]],c("meanInfRV", "log10mean", "log2FC", "qvalue")]
# txps <- intersect(unlist(rowData(yg)[gs,"tx_ids"]), rownames(y))
# mcols(y)[txps, c("meanInfRV", "log10mean", "log2FC", "qvalue")]
# treeCons$tip[unlist(Descendants(treeCons, treeDE[[2]][lfcNodes[j]]))]

# j <- 12
# gs <- genesTreeDE[[2]][[lfcNodes[j]]]
# print(gs)
# mcols(yg)[gs,c("log10mean", "log2FC", "qvalue")]
# mcols(yAll)[treeDE[[2]][lfcNodes[j]],c("meanInfRV", "log10mean", "log2FC", "qvalue")]
# txps <- intersect(unlist(rowData(yg)[gs,"tx_ids"]), rownames(y))
# mcols(y)[txps, c("meanInfRV", "log10mean", "log2FC", "qvalue")]
# treeCons$tip[unlist(Descendants(treeCons, treeDE[[2]][lfcNodes[j]]))]

[1] "ENSPTRG00000017289"


DataFrame with 1 row and 3 columns
                   log10mean    log2FC    qvalue
                   <numeric> <numeric> <numeric>
ENSPTRG00000017289   2.92595  0.200591  0.478691

DataFrame with 1 row and 4 columns
          meanInfRV log10mean    log2FC      qvalue
          <numeric> <numeric> <numeric>   <numeric>
Node47106  0.121311   2.87189  0.430705 1.09745e-06

DataFrame with 3 rows and 4 columns
                     meanInfRV log10mean    log2FC    qvalue
                     <numeric> <numeric> <numeric> <numeric>
ENSPTRT00000031995.4 19.795755   2.63370 0.3996194  0.194952
ENSPTRT00000077169.1  0.203219   1.66853 0.0648727  0.790782
ENSPTRT00000083497.1 29.756853   2.49691 0.4781580  0.365346

[1] "ENSPTRT00000083497.1" "ENSPTRT00000031995.4"

...

[1] "ENSPTRG00000007533"


DataFrame with 1 row and 3 columns
                   log10mean    log2FC    qvalue
                   <numeric> <numeric> <numeric>
ENSPTRG00000007533   2.72469 -0.499219   0.22207

DataFrame with 1 row and 4 columns
          meanInfRV log10mean    log2FC     qvalue
          <numeric> <numeric> <numeric>  <numeric>
Node48252 0.0731386   2.78153 -0.501752 0.00193015

DataFrame with 3 rows and 4 columns
                     meanInfRV log10mean    log2FC    qvalue
                     <numeric> <numeric> <numeric> <numeric>
ENSPTRT00000013883.5  12.09973   2.30319 -0.889919  0.136598
ENSPTRT00000079623.1  12.86019   2.31672 -0.439393  0.256928
ENSPTRT00000100721.1   8.33453   2.29293 -0.351177  0.201557

[1] "ENSPTRT00000079623.1" "ENSPTRT00000013883.5" "ENSPTRT00000100721.1"

Transcript end side different
https://useast.ensembl.org/Mus_musculus/Gene/Summary?db=core;g=ENSMUSG00000028936;r=4:152410199-152418528 
Mutually exclusive exon(start)
https://useast.ensembl.org/Mus_musculus/Gene/Summary?db=core;g=ENSMUSG00000045917;r=4:63477018-63504594
Transcript end side different
https://useast.ensembl.org/Mus_musculus/Gene/Summary?db=core;g=ENSMUSG00000030678;r=7:126621302-126626209
Different start side
https://useast.ensembl.org/Mus_musculus/Gene/Summary?db=core;g=ENSMUSG00000058881;r=18:82928788-83023439
Skipped Exon
https://useast.ensembl.org/Mus_musculus/Gene/Summary?db=core;g=ENSMUSG00000002546;r=2:32177396-32197933

#### Genes belonging to treeDE nodes that intersect with differentially expressed genes but are not covered by transcripts.'
We see many such genes that contain differential nodes but no differentially expressed transcripts. The magnitude of logFC in most cases though decreases from trancript to gene.

In [17]:
genesNodeCommonDGE <- lapply(seq_along(diffGenes), function(i) unlist(genesTreeDE[[i]])[unlist(genesTreeDE[[i]]) %in% diffGenes[[i]]]) ## Number of genes mapping to inner nodes that also map to dges but not dtes
                             
#### Extracting the index in geneTreeDE                      
treeDECommonInds <- lapply(seq_along(genesNodeCommonDGE), function(i) {
    sapply(genesNodeCommonDGE[[i]], function(genes) {
        sapply(genes, function(gene) {
            which(sapply(genesTreeDE[[i]], function(gs) sum(gene %in% gs) > 0))
        })
    })
})
sapply(genesNodeCommonDGE, length)

[1] 1217  151  131

In [18]:
#### Extracting the gene nodes that are unique aka they map to only 1 gene
#### Note that single gene can map to multiple nodes within treeDE, so we need to remove that at well
treeDECommonInds <- lapply(seq_along(treeDECommonInds), function(i) {
    iList <- treeDECommonInds[[i]]
    inds <- which(sapply(iList, function(iL) {
        length(unique(unlist(genesTreeDE[[i]][iL])))==1
    }))
    iList[names(iList)[inds]]
})
sapply(treeDECommonInds, length)

[1] 1180  146  128

In [62]:
head(order(mcols(gy)[names(treeDECommonInds[[2]]), "qvalue"]),30)

[1]  23  43  62  73  74  97  98 101 102 127 136  14  69 140   9 125  81  93  28
[20]  68 138  18  94  35  40 122 105 119  57  55

In [53]:
for(j in c(73, 97, 136, 69, 9, 93)) {
        gs <- names(treeDECommonInds[[2]])[j]
        print(gs)
        print(mcols(gy)[gs,c("log10mean", "log2FC", "qvalue")])
        node <- treeDE[[2]][treeDECommonInds[[2]][[j]]]
        print(mcols(yAll)[node,c("meanInfRV", "log10mean", "log2FC", "qvalue")])
        txps <- intersect(rnames[unlist(rowData(gy)[gs,"tx_ids"])], rownames(y))
        print(mcols(y)[txps, c("meanInfRV", "log10mean", "log2FC", "qvalue")])
        print(treeCons$tip[unlist(Descendants(treeCons, node))])
        print("next")
}


[1] "ENSPTRG00000017182"
DataFrame with 1 row and 3 columns
                   log10mean    log2FC      qvalue
                   <numeric> <numeric>   <numeric>
ENSPTRG00000017182   2.74239 -0.511141 4.19463e-06
DataFrame with 2 rows and 4 columns
          meanInfRV log10mean    log2FC     qvalue
          <numeric> <numeric> <numeric>  <numeric>
Node57641   3.68507   2.04386 -0.599623 0.04828378
Node57643   0.36223   2.69548 -0.393650 0.00193015
DataFrame with 6 rows and 4 columns
                     meanInfRV log10mean    log2FC    qvalue
                     <numeric> <numeric> <numeric> <numeric>
ENSPTRT00000061959.3   7.77064   1.30743        NA        NA
ENSPTRT00000080856.1  25.46061   2.09580 -1.245293 0.0893939
ENSPTRT00000093587.1   8.14137   2.55154 -0.153924 0.3663176
ENSPTRT00000102108.1   6.46997   1.46636 -0.479068 0.4863730
ENSPTRT00000103985.1   5.06890   1.92007 -0.430987 0.3432334
ENSPTRT00000106380.1  11.16375   1.24887        NA        NA
[1] "ENSPTRT00000103985

https://useast.ensembl.org/Pan_troglodytes/Gene/Summary?db=core;g=ENSPTRG00000017182;r=5:123343707-123449338 (missing exons)
https://useast.ensembl.org/Pan_troglodytes/Gene/Summary?db=core;g=ENSPTRG00000008167;r=16:48444105-48488981 (different exons)
https://useast.ensembl.org/Pan_troglodytes/Gene/Summary?db=core;g=ENSPTRG00000011776;r=2A:28200362-28222512 (skipped exons)
https://useast.ensembl.org/Pan_troglodytes/Gene/Summary?db=core;g=ENSPTRG00000008182;r=16:49204939-49210491
(Different exons)
https://useast.ensembl.org/Pan_troglodytes/Gene/Summary?db=core;g=ENSPTRG00000008182;r=16:49204939-49210491
(Skipped exons)